In [5]:
import os 
os.getcwd()
os.chdir(r'C:\Users\MAIN\Desktop\ML\Internship SMinds\Recommender')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
#import dataset
df = pd.read_csv("TripAdvReview.csv", na_values=" ")
df.head(n=5)
df.columns
#data wrangling
print('Unique counts:',df.nunique())

print('Kind of ratings:',df.Rating.unique())
#remove extra columns and keep the necessary ones for analysis
df = pd.DataFrame(df.drop(['Bubble_Count', 'Review_Count'], axis=1))
df.head(n=5)
df.columns
#
df.describe()
df.shape
df.size
df.isnull().sum()

Unique counts: Title            846
Bubble_Count       7
Rating             7
Review_Count      70
Review_counts     69
City             327
Province          12
Reviews          570
dtype: int64
Kind of ratings: [5.  4.5 4.  3.5 nan 1.  2.  3. ]


Title              0
Rating           221
Review_counts    281
City               0
Province           0
Reviews          278
dtype: int64

In [7]:
#remove new line characters from variable columns
df = df.replace(r'\n',' ', regex=True)

#remove the numbers in the review column
df.Reviews = df.Reviews.str.replace('\d+', '')

In [8]:
#fills the rating and Review_counts variable missing values with the mean and median respectively
df = df.fillna({'Rating': df.Rating.median(), 'Review_counts': df.Review_counts.mean()})

In [9]:
df.isnull().sum()

Title              0
Rating             0
Review_counts      0
City               0
Province           0
Reviews          278
dtype: int64

In [10]:
#drop all missing values
df.dropna(axis=0,how='any', inplace= True)
df.head(n=100)

,Title,Rating,Review_counts,City,Province,Reviews
0,Vacation/Cottage Rental,5.0,85.0,Niagara Falls,Ontario,mention of vacation rentals This rental felt ...
1,Mistiso's Place Vacation Rentals- Purcell Suite,4.5,100.0,Nelson,British Columbia,mentions of vacation rentals I can't say enou...
2,Toronto 4 Bedroom House Spacious Clean Beautif...,5.0,36.0,Toronto,Ontario,mention of vacation rentals back yard! Would ...
3,Our Sweet Escape,5.0,34.0,Qualicum Beach,British Columbia,mentions of vacation rentals enjoyable. I hav...
4,Walk to ocean from Lyons Cottage Rentals in PEI,4.5,48.0,Stanhope,Prince Edward Island,mentions of vacation rentals quiet morning as...
5,Best views in Nelson,5.0,33.0,Nelson,British Columbia,mentions of vacation rentals to buy even the ...
6,"Shale Beach House, Blue Mountain Collingwood Ont.",5.0,77.0,Blue Mountains,Ontario,mention of vacation rentals vacation spot! Th...
7,Paradise Vacation Rentals,5.0,25.0,North Bay,Ontario,This has to be one of the best vacation places...
8,Downtown Toronto 4BR Townhouse,5.0,55.0,Toronto,Ontario,mentions of vacation rentals paint but you co...
9,Les Rives du Sanctuaire,5.0,53.0,Mont Tremblant,Quebec,mentions of vacation rentals spend majestic v...


In [11]:
df.index=range(571)

In [12]:
print(df.shape)
df.isnull().sum()

(571, 6)


Title            0
Rating           0
Review_counts    0
City             0
Province         0
Reviews          0
dtype: int64

In [13]:
# Let's understand the two lists: reviews (text_train) and their labels (y_train)
print("Type : ",type(df.Reviews))
print("Length of reviews: ",len(df.Reviews))
print("Review at index 6:\n ", df.Reviews[6])
print("Label of the review at Index 6: ",df.Rating[6])
# The ratings laels is a continous series of float numbers

Type :  <class 'pandas.core.series.Series'>
Length of reviews:  571
Review at index 6:
   mention of vacation rentals vacation spot! There are three levels and so each family got their very own
Label of the review at Index 6:  5.0


In [14]:
#import libraries
import nltk
import re
from nltk.corpus import stopwords

In [15]:
#filter punctuations, stemming and stopwords
corpus = []
for i in range(len(df)):
    review = re.sub('[^a-zA-Z0-9]', ' ', df['Reviews'][i])
    review = review.lower()
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    stemmer = nltk.PorterStemmer()
    lemmatizer = nltk.WordNetLemmatizer()
    tokens_list = tokenizer.tokenize(review)
    tokens = []
    for token in tokens_list:
        tokens.append(stemmer.stem(lemmatizer.lemmatize(token)))
        stop_words = stopwords.words("english")
    filtered_words = [w for w in tokens if w not in stop_words]
    review = ' '.join(filtered_words)
    corpus.append(review)


In [16]:
#Bag of Words model to convert corpus into X
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cv.fit(corpus)
key = list(cv.vocabulary_.keys())
key.sort()
X = pd.DataFrame(cv.transform(corpus).toarray(),columns = key)
y = df.Rating

#TF_IDF model to convert corpus into X
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

X2 = pd.DataFrame(tfidf.fit_transform(corpus).toarray())


In [17]:
#Label Encoding for Categorical Target Variable

from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
df['Rating'] = lb.fit_transform(df['Rating'])
y = df.Rating

In [19]:
#logistic Regression model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)


#import classifier
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

clf.fit(X_train,y_train)

weights = clf.coef_

print(clf.score(X_test,y_test))

#predictions for test
y_pred1 = clf.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,clf.predict(X_test))
print(cm)


0.6643356643356644
[[ 0  0  0  0  1]
 [ 0  0  0  0  3]
 [ 0  0  0  2  7]
 [ 0  0  0  4 25]
 [ 0  0  0 10 91]]


C:\Users\MAIN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MAIN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [20]:
#Random Forest Model
#import classifier
from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier()

clf1.fit(X_train,y_train)

#weights = clf.coef_

y_pred2 = clf1.predict(X_test)

print(clf1.score(X_test,y_test))

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,clf1.predict(X_test))
print(cm)

0.6643356643356644
[[ 0  0  0  0  1]
 [ 0  0  0  0  3]
 [ 0  0  0  0  9]
 [ 0  0  0  2 27]
 [ 0  0  0  8 93]]


C:\Users\MAIN\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [21]:
#Adaboost Model
#import classifier
from sklearn.ensemble import AdaBoostClassifier
clf2 = AdaBoostClassifier()

clf2.fit(X_train,y_train)

#weights = clf2.coef_

y_pred3 = clf2.predict(X_test)

print(clf2.score(X_test,y_test))

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,clf2.predict(X_test))
print(cm)

0.6923076923076923
[[ 0  0  0  0  1]
 [ 0  0  0  0  3]
 [ 0  0  0  0  9]
 [ 0  0  0  0 29]
 [ 2  0  0  0 99]]


In [22]:
#KNearest Neighbor Model
#import classifier
from sklearn.neighbors import KNeighborsClassifier
clf3 = KNeighborsClassifier()

clf3.fit(X_train,y_train)

#weights = clf.coef_

y_pred4 = clf3.predict(X_test)

print(clf3.score(X_test,y_test))

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,clf3.predict(X_test))
print(cm)

0.6013986013986014
[[ 0  0  0  1  0]
 [ 0  0  1  0  2]
 [ 0  0  0  3  6]
 [ 0  0  5  2 22]
 [ 0  0  9  8 84]]


In [23]:
#Support Vector Machine(Classifier) Model 
#import classifier
from sklearn.svm import SVC
clf4 = SVC(kernel='linear')

clf4.fit(X_train,y_train)

#weights = clf.coef_

y_pred5 = clf4.predict(X_test)

print(clf4.score(X_test,y_test))

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,clf4.predict(X_test))
print(cm)

0.5874125874125874
[[ 0  0  0  0  1]
 [ 0  0  0  0  3]
 [ 0  0  0  2  7]
 [ 0  0  2  5 22]
 [ 0  0  0 22 79]]
